In [1]:
import json
import pandas as pd

In [2]:
with open("vns_benchmarks_300s.json", "r", encoding="utf-8") as f:
    vns_300s = json.load(f)

with open("gurobi_benchmarks.json", "r", encoding="utf-8") as f:
    gurobi_60s = json.load(f)

vns_300s: dict; gurobi_60s: dict

print(vns_300s.keys() == gurobi_60s.keys())


True


In [3]:
project_quantities = [3, 4, 5]
student_quantities = [30, 40, 50]

def get_benchmarking_data_by_dimension(benchmarking_data: dict):
    benchmarking_data_by_dimension = {}
    for project_quantity in project_quantities:
        benchmarking_data_by_dimension[project_quantity] = {}
        for student_quantity in student_quantities:
            benchmarking_data_by_dimension[project_quantity][student_quantity] = {}
    
    for instance, solutions in benchmarking_data.items():
        for project_quantity in project_quantities:
            for student_quantity in student_quantities:
                if f"_{project_quantity}_{student_quantity}" in instance:
                    benchmarking_data_by_dimension[project_quantity][student_quantity][int(instance[-1])] = solutions
    return benchmarking_data_by_dimension

vns_300s_dimensions = get_benchmarking_data_by_dimension(vns_300s)
gurobi_60s_dimensions = get_benchmarking_data_by_dimension(gurobi_60s)

In [4]:
def round_minor_deviations(x: float) -> float | int:
    nearest = round(x)
    if abs(x - nearest) <= 1e-6:
        return nearest
    return x


def best_solution_whithin_time_limit(solutions, time_limit=300):
    for solution in reversed(solutions):
        if solution["runtime"] <= time_limit:
            return solution


results_dimensions = {}
for project_quantity in project_quantities:
    results_dimensions[project_quantity] = {}
    for student_quantity in student_quantities:
        vns_300s_dimension = vns_300s_dimensions[project_quantity][student_quantity]
        vns_300s_dimension: dict
        results_dimensions[project_quantity][student_quantity] = {}
        results_dimension = results_dimensions[project_quantity][student_quantity]
        results_dimension["granular"] = pd.DataFrame.from_dict(
            {
                instance: {
                    # "Instance": instance,
                    "Best VNS": (best_vns := int((solution := best_solution_whithin_time_limit(solutions))["obj"])),
                    "Runtime (s)": float(solution["runtime"]),
                    "Best Gurobi": (
                        best_gurobi := int(
                            gurobi_60s_dimensions[project_quantity][student_quantity][instance]["best_obj"]
                        )
                    ),
                    "Upper Bound": (
                        upper_bound_gurobi := round_minor_deviations(
                            gurobi_60s_dimensions[project_quantity][student_quantity][instance]["obj_bound"]
                        )
                    ),
                    "Gap VNS to Gurobi": (best_gurobi - best_vns) / best_gurobi,
                    "Gap Gurobi": (upper_bound_gurobi - best_gurobi) / upper_bound_gurobi,
                }
                for instance, solutions in vns_300s_dimension.items()
            },
            orient="index",
        )

        results_dimension_granular = results_dimension["granular"]
        results_dimension_granular: pd.DataFrame

        means = results_dimension_granular[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].mean()
        medians = results_dimension_granular[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].median()
        maximum = results_dimension_granular[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].max()
        minimum = results_dimension_granular[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].min()
        results_dimension["summary"] = pd.DataFrame([means, medians, maximum, minimum], index=["Mean", "Median", "Maximum", "Minimum"])
        results_dimension_summary = results_dimension["summary"]
        results_dimension_summary: pd.DataFrame

        results_dimension_summary_presentation = results_dimension_summary.copy()
        results_dimension_summary_presentation[["Gap VNS to Gurobi", "Gap Gurobi"]] = results_dimension_summary_presentation[
            ["Gap VNS to Gurobi", "Gap Gurobi"]
        ].map("{:.2%}".format)
        results_dimension_summary_presentation[["Runtime (s)"]] = results_dimension_summary_presentation[["Runtime (s)"]].map(
            "{:.2f}".format
        )
        results_dimension["summary_presentation"] = results_dimension_summary_presentation

        results_dimension_presentation = results_dimension_granular.copy()
        results_dimension_presentation[["Best VNS", "Best Gurobi", "Upper Bound"]] = results_dimension_presentation[
            ["Best VNS", "Best Gurobi", "Upper Bound"]
        ].map("{:.0f}".format)
        results_dimension_presentation[["Gap VNS to Gurobi", "Gap Gurobi"]] = results_dimension_presentation[
            ["Gap VNS to Gurobi", "Gap Gurobi"]
        ].map("{:.2%}".format)
        results_dimension_presentation[["Runtime (s)"]] = results_dimension_presentation[["Runtime (s)"]].map(
            "{:.2f}".format
        )
        results_dimension_presentation.index.name = "Instance"
        results_dimension["granular_presentation"] = results_dimension_presentation

In [5]:
results_dimensions[3][30]["granular_presentation"]


,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,79,0.61,81,81,2.47%,0.00%
1,92,3.38,92,92,0.00%,0.00%
2,86,10.27,86,86,0.00%,0.00%
3,95,34.69,98,102,3.06%,3.92%
4,99,42.86,101,101,1.98%,0.00%
5,96,8.16,96,96,0.00%,0.00%
6,98,143.89,98,98,0.00%,0.00%
7,69,7.44,71,75,2.82%,5.33%
8,100,82.02,101,104,0.99%,2.88%


In [6]:
results_dimensions[3][30]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,33.71,1.26%,2.11%
Median,9.22,1.11%,0.00%
Maximum,143.89,3.06%,8.99%
Minimum,0.61,0.00%,0.00%


In [7]:
results_dimensions[3][40]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,117,66.26,118,118,0.85%,0.00%
1,105,78.76,112,119,6.25%,5.88%
2,114,5.73,116,123,1.72%,5.69%
3,113,9.19,114,114,0.88%,0.00%
4,121,87.80,123,134,1.63%,8.21%
5,115,2.27,122,130,5.74%,6.15%
6,102,35.69,102,109,0.00%,6.42%
7,103,129.34,105,115,1.90%,8.70%
8,119,16.27,121,124,1.65%,2.42%


In [8]:
results_dimensions[3][40]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,43.33,2.06%,4.52%
Median,25.98,1.64%,5.79%
Maximum,129.34,6.25%,8.70%
Minimum,2.02,0.00%,0.00%


In [9]:
results_dimensions[3][50]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,155,226.52,160,164,3.12%,2.44%
1,151,72.09,154,159,1.95%,3.14%
2,138,270.20,139,146,0.72%,4.79%
3,120,22.61,123,133,2.44%,7.52%
4,83,210.54,88,93,5.68%,5.38%
5,144,269.06,147,159,2.04%,7.55%
6,131,208.22,133,139,1.50%,4.32%
7,150,42.10,154,165,2.60%,6.67%
8,130,12.69,136,140,4.41%,2.86%


In [10]:
results_dimensions[3][50]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,140.85,2.83%,5.24%
Median,141.33,2.52%,5.09%
Maximum,270.20,5.68%,7.75%
Minimum,12.69,0.72%,2.44%


In [11]:
results_dimensions[4][30]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,89,149.37,89,89,0.00%,0.00%
1,92,54.04,96,96,4.17%,0.00%
2,103,87.72,102,109,-0.98%,6.42%
3,103,16.38,104,104,0.96%,0.00%
4,106,1.13,111,116,4.50%,4.31%
5,88,82.75,89,94,1.12%,5.32%
6,111,259.40,114,115,2.63%,0.87%
7,90,46.88,95,95,5.26%,0.00%
8,84,1.89,87,87,3.45%,0.00%


In [12]:
results_dimensions[4][30]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,70.06,2.53%,2.28%
Median,50.46,3.04%,0.43%
Maximum,259.40,5.26%,6.42%
Minimum,1.06,-0.98%,0.00%


In [13]:
results_dimensions[4][40]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,128,258.47,129,135,0.78%,4.44%
1,115,13.38,118,128,2.54%,7.81%
2,124,6.00,123,137,-0.81%,10.22%
3,115,202.73,116,122,0.86%,4.92%
4,113,103.85,116,116,2.59%,0.00%
5,114,243.37,118,130,3.39%,9.23%
6,118,121.02,121,127,2.48%,4.72%
7,121,5.55,126,126,3.97%,0.00%
8,130,74.44,135,139,3.70%,2.88%


In [14]:
results_dimensions[4][40]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,110.73,2.10%,4.99%
Median,91.14,2.51%,4.82%
Maximum,258.47,3.97%,10.22%
Minimum,5.55,-0.81%,0.00%


In [15]:
results_dimensions[4][50]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,153,121.01,159,166,3.77%,4.22%
1,144,20.34,148,159,2.70%,6.92%
2,158,54.31,160,160,1.25%,0.00%
3,145,196.18,150,160,3.33%,6.25%
4,168,50.92,170,176,1.18%,3.41%
5,147,14.05,153,153,3.92%,0.00%
6,153,83.18,158,171,3.16%,7.60%
7,153,166.22,156,161,1.92%,3.11%
8,153,20.55,160,173,4.38%,7.51%


In [16]:
results_dimensions[4][50]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,101.70,2.77%,4.42%
Median,68.74,2.93%,4.72%
Maximum,290.29,4.38%,7.60%
Minimum,14.05,1.18%,0.00%


In [17]:
results_dimensions[5][30]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,94,1.89,97,97,3.09%,0.00%
1,89,102.87,94,101,5.32%,6.93%
2,100,66.31,102,112,1.96%,8.93%
3,107,42.12,110,115,2.73%,4.35%
4,92,71.30,103,103,10.68%,0.00%
5,98,184.87,100,100,2.00%,0.00%
6,84,64.73,87,87,3.45%,0.00%
7,92,150.69,94,101,2.13%,6.93%
8,102,4.95,103,108,0.97%,4.63%


In [18]:
results_dimensions[5][30]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,71.20,3.45%,3.18%
Median,65.52,2.44%,2.17%
Maximum,184.87,10.68%,8.93%
Minimum,1.89,0.97%,0.00%


In [19]:
results_dimensions[5][40]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,132,5.82,138,147,4.35%,6.12%
1,133,21.97,137,144,2.92%,4.86%
2,133,114.83,135,142,1.48%,4.93%
3,132,13.39,135,135,2.22%,0.00%
4,125,12.03,128,133,2.34%,3.76%
5,130,11.09,132,135,1.52%,2.22%
6,134,266.79,138,145,2.90%,4.83%
7,129,33.14,133,138,3.01%,3.62%
8,129,62.77,130,134,0.77%,2.99%


In [20]:
results_dimensions[5][40]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,69.25,2.68%,3.76%
Median,27.56,2.62%,4.00%
Maximum,266.79,5.31%,6.12%
Minimum,5.82,0.77%,0.00%


In [21]:
results_dimensions[5][50]["granular_presentation"]

,Best VNS,Runtime (s),Best Gurobi,Upper Bound,Gap VNS to Gurobi,Gap Gurobi
Instance,,,,,,
0,161,97.75,170,183,5.29%,7.10%
1,143,22.80,151,170,5.30%,11.18%
2,169,52.35,170,184,0.59%,7.61%
3,166,27.42,169,178,1.78%,5.06%
4,162,169.56,167,175,2.99%,4.57%
5,158,48.67,162,171,2.47%,5.26%
6,145,15.97,152,167,4.61%,8.98%
7,144,266.33,147,150,2.04%,2.00%
8,168,23.62,170,175,1.18%,2.86%


In [22]:
results_dimensions[5][50]["summary_presentation"]

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,74.46,2.70%,6.97%
Median,38.05,2.25%,6.18%
Maximum,266.33,5.30%,15.03%
Minimum,15.97,0.59%,2.00%


In [23]:
results_all_instances = pd.DataFrame.from_dict({
    instance: {
        "Best VNS": (best_vns := int((solution := best_solution_whithin_time_limit(solutions))["obj"])),
        "Runtime (s)": solution["runtime"],
        "Best Gurobi": (best_gurobi := int(gurobi_60s[instance]["best_obj"])),
        "Upper Bound": (upper_bound_gurobi := round_minor_deviations(gurobi_60s[instance]["obj_bound"])),
        "Gap VNS to Gurobi": (best_gurobi - best_vns) / best_gurobi,
        "Gap Gurobi": (upper_bound_gurobi - best_gurobi) / upper_bound_gurobi,
    }
    for instance, solutions in vns_300s.items()
}, orient="index")

means = results_all_instances[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].mean()
medians = results_all_instances[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].median()
maximum = results_all_instances[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].max()
minimum = results_all_instances[["Runtime (s)", "Gap VNS to Gurobi", "Gap Gurobi"]].min()
summary_stats_all_instances = pd.DataFrame([means, medians, maximum, minimum], index=["Mean", "Median", "Maximum", "Minimum"])


In [24]:
summary_stats_all_instances_presentation = summary_stats_all_instances.copy()
summary_stats_all_instances_presentation[["Gap VNS to Gurobi", "Gap Gurobi"]] = (
    summary_stats_all_instances_presentation[["Gap VNS to Gurobi", "Gap Gurobi"]].map("{:.2%}".format)
)
summary_stats_all_instances_presentation[["Runtime (s)"]] = summary_stats_all_instances_presentation[
    ["Runtime (s)"]
].map("{:.2f}".format)

summary_stats_all_instances_presentation

,Runtime (s),Gap VNS to Gurobi,Gap Gurobi
Mean,79.48,2.49%,4.16%
Median,51.64,2.28%,4.40%
Maximum,290.29,10.68%,15.03%
Minimum,0.61,-0.98%,0.00%
